In [10]:
# ============================================================
# train_model_v11.py
#  - FE_v11(segment + norm_diff_corr) 기반 모델링
#  - A의 시계열성 + 관계 Feature로 B의 value 예측
# ============================================================

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from pathlib import Path

# ============================================
# 0. PATH
# ============================================
BASE_DIR = Path.cwd().resolve()
DATA_DIR = BASE_DIR.parents[1] / "data"

PAIR_FE_PATH    = "../preprocessing/pair_features_v11_segment.csv"
MONTHLY_V6_PATH = DATA_DIR / "processed" / "train_monthly_v6.csv"

print("📂 DATA_DIR:", DATA_DIR)
print("📄 FE file:", PAIR_FE_PATH)

# ============================================
# 1. LOAD DATA
# ============================================
pair = pd.read_csv(PAIR_FE_PATH)
print("🟢 pair_features_v11:", pair.shape)

monthly = pd.read_csv(MONTHLY_V6_PATH)

# ============================================
# 2. TARGET: B(item_j)의 마지막 관측 값
# ============================================
last_year  = monthly["year"].max()
last_month = monthly[monthly["year"]==last_year]["month"].max()

print(f"🔎 마지막 관측 월 → target: {last_year}-{last_month}")

target_month = monthly[
    (monthly["year"]  == last_year) &
    (monthly["month"] == last_month)
]

target_map = dict(zip(target_month["item_id"], target_month["total_value"]))
pair["target"] = pair["item_j"].map(target_map).astype(float).fillna(0)

print("🎯 target>0 개수:", (pair["target"] > 0).sum())
print("🎯 target unique:", pair["target"].nunique())

# ============================================
# 3. A/B 마지막 시점 feature merge
# ============================================

last_t = monthly["t"].max()
last_rows = monthly[monthly["t"] == last_t].copy()

item_feats = last_rows[[
    "item_id",
    "smooth_value",
    "smooth_value_mean_3",
    "smooth_value_mean_6",
    "slope_3",
    "slope_6",
]]

item_feats = item_feats.rename(columns={
    "smooth_value":        "smooth_last",
    "smooth_value_mean_3": "mean3_last",
    "smooth_value_mean_6": "mean6_last",
    "slope_3":             "slope3_last",
    "slope_6":             "slope6_last",
})

# A merge
pair = pair.merge(
    item_feats.add_prefix("A_").rename(columns={"A_item_id":"item_i"}),
    on="item_i", how="left"
)

# B merge
pair = pair.merge(
    item_feats.add_prefix("B_").rename(columns={"B_item_id":"item_j"}),
    on="item_j", how="left"
)

num_cols = [c for c in pair.columns if c not in ["item_i","item_j"]]
pair[num_cols] = pair[num_cols].replace([np.inf,-np.inf], np.nan).fillna(0)

# ============================================
# 4. FEATURES (v11 완전 싱크)
# ============================================
RELATION_FEATS = [
    "hs4_same", "hs3_same", "hs2_same", "hs_distance",
    "rolling_corr3", "rolling_corr6",
    "seg_max_corr", "seg_mean_pos_corr", "seg_high_cnt",
    "norm_diff_corr",          # ★ NEW: FE_v11 추가
    "best_lag", "lag_corr",
    "scale_ratio",
]

A_FEATS = [
    "A_smooth_last",
    "A_mean3_last",
    "A_mean6_last",
    "A_slope3_last",
    "A_slope6_last",
]

B_FEATS = [
    "B_smooth_last",
    "B_mean3_last",
    "B_mean6_last",
    "B_slope3_last",
    "B_slope6_last",
]

FEATURES = RELATION_FEATS + A_FEATS + B_FEATS

missing = [c for c in FEATURES if c not in pair.columns]
if missing:
    raise ValueError(f"❌ 누락된 FE 컬럼: {missing}")

pair[FEATURES] = pair[FEATURES].astype(float).fillna(0)

# ============================================
# 5. Stratified bin + sample weight
# ============================================
pair["target_clip"] = pair["target"].clip(lower=0)
pair["target_log"]  = np.log1p(pair["target_clip"])

if pair["target_clip"].nunique() > 1:
    ranked = pair["target_log"].rank(method="first")
    pair["target_bin"] = pd.qcut(ranked, q=4, labels=False, duplicates="drop")
else:
    pair["target_bin"] = 0

pair["target_bin"] = pair["target_bin"].fillna(0).astype(int)
pair["sample_weight"] = np.where(pair["target_clip"]>0, 1.0, 0.5)

print("📊 stratify bins:")
print(pair["target_bin"].value_counts().sort_index())

# ============================================
# 6. LightGBM Training
# ============================================
def train_single_model(seed):
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    oof = np.zeros(len(pair))
    models = []

    for fold, (tr_idx, va_idx) in enumerate(kfold.split(pair, pair["target_bin"])):

        tr = pair.iloc[tr_idx]
        va = pair.iloc[va_idx]

        model = LGBMRegressor(
            objective="regression",

            max_depth = -1,
            num_leaves = 127,

            subsample = 0.85,
            colsample_bytree = 0.85,

            min_child_samples = 20,
            reg_alpha  = 0.1,
            reg_lambda = 0.3,

            learning_rate = 0.02,
            n_estimators  = 2500,

            random_state = seed,
            n_jobs       = -1,
        )

        model.fit(
            tr[FEATURES],
            tr["target_log"],
            sample_weight=tr["sample_weight"],
            eval_set=[(va[FEATURES], np.log1p(va["target_clip"]))],
            eval_metric="rmse",
        )

        pred_log = model.predict(va[FEATURES])
        pred_val = np.expm1(pred_log)
        pred_val = np.clip(pred_val, 0, None)

        oof[va_idx] = pred_val
        models.append(model)

        rmse = np.sqrt(mean_squared_error(va["target_clip"], pred_val))
        print(f"   Fold {fold} RMSE = {rmse:.4f}")

    rmse_all = np.sqrt(mean_squared_error(pair["target_clip"], oof))
    print(f"🔥 Seed {seed} OOF RMSE = {rmse_all:.5f}")

    return models, oof

SEEDS = [42, 2025, 314159]
all_models = []
all_oof = np.zeros(len(pair))

for sd in SEEDS:
    print(f"\n🚀 Training seed = {sd}")
    models, oof = train_single_model(sd)
    all_models += models
    all_oof += oof / len(SEEDS)

final_rmse = np.sqrt(mean_squared_error(pair["target_clip"], all_oof))
print(f"\n✅ Final Ensemble OOF RMSE = {final_rmse:.5f}")

# ============================================
# 7. Final submission
# ============================================
X_test = pair[FEATURES].values

preds_log = np.mean([m.predict(X_test) for m in all_models], axis=0)
pred_val = np.expm1(preds_log)
pred_val = np.clip(pred_val, 0, None)

submission = pd.DataFrame({
    "leading_item_id": pair["item_i"],
    "following_item_id": pair["item_j"],
    "value": pred_val.round().astype(int),
})

save_path = "v7_submission_LGBM_segment_only.csv"
submission.to_csv(save_path, index=False)

print("\n🎉 Final submission saved:", save_path)
print("📌 제출 행 수:", len(submission))
print(submission.head())


📂 DATA_DIR: /data/ephemeral/home/data
📄 FE file: ../preprocessing/pair_features_v11_segment.csv
🟢 pair_features_v11: (118, 15)
🔎 마지막 관측 월 → target: 2025-7
🎯 target>0 개수: 115
🎯 target unique: 48
📊 stratify bins:
target_bin
0    30
1    29
2    29
3    30
Name: count, dtype: int64

🚀 Training seed = 42
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000080 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 491
[LightGBM] [Info] Number of data points in the train set: 94, number of used features: 19
[LightGBM] [Info] Start training from score 12.287357
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/data/ephemeral/home/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with featur